# Import & Load Safety information

### Config

In [1]:
import sys, os
import configparser
config = configparser.ConfigParser()
config.read_file(open('../../settings.ini'))

sys.path.insert(0, config.get('PATHS','libs_path'))
engine_path = config.get('DATABASE','engine_path')

### Main params

In [2]:
remote_path = 'http://pacha.datawheel.us/'
local_path = '../data/'

### Imports

In [3]:
import postgres #from local file postgres.py
from commons import inline_table_xml, download_file, download_zip_file, extract_zip_file #from local file commons.py

import json
import pandas as pd
from sqlalchemy import create_engine

### Load file

In [4]:
download_zip_file(remote_path,
                  local_path,
                  "crime.zip")

extract_zip_file(local_path, "temp.zip")

Downloading... http://pacha.datawheel.us/crime.zip
Unzipping... ../data/temp.zip


True

In [5]:
df = pd.read_csv(os.path.join(local_path, "felonies.csv"),low_memory=True)

In [6]:
list(df)

['year', 'comuna_datachile_id', 'felony_id', 'cases']

In [7]:
df.rename(columns={'comuna_datachile_id': 'comuna_id'}, inplace=True)
df = df.astype({'year':'int','comuna_id':'int','felony_id':'int','cases':'int'})
list(df)

['year', 'comuna_id', 'felony_id', 'cases']

### Ingest

In [8]:
engine = create_engine(engine_path)
db = postgres.PostgresDriver(engine)
db.to_sql(df, 'safety', 'fact_crimes')

DROP TABLE IF EXISTS safety.fact_crimes;
CREATE TABLE "safety"."fact_crimes" (
"year" INTEGER,
  "comuna_id" INTEGER,
  "felony_id" INTEGER,
  "cases" INTEGER
)
COPY "safety"."fact_crimes" ("year","comuna_id","felony_id","cases") FROM STDIN WITH CSV HEADER DELIMITER ',';


### Indexes & FK

In [9]:
engine.execute("""
CREATE INDEX fact_crimes_comuna_id
ON safety.fact_crimes (comuna_id)
""")

### Related dim felony

In [10]:
felonies = pd.read_csv(os.path.join(local_path, "categories_id.csv"),low_memory=True)
print (inline_table_xml(felonies, 'felony_es', 'felony_id', 'felony_es'))


<InlineTable alias="felony_es">
  <ColumnDefs>
    <ColumnDef name="id" type="Numeric" />
    <ColumnDef name="description" type="String" />
    <ColumnDef name="es_description" type="String" />
  </ColumnDefs>
  <Rows>
    <Row>
      <Value column="id">1</Value>
      <Value column="description">robo con violencia o intimidación</Value>
      <Value column="es_description">robo con violencia o intimidación</Value>
    </Row>
  <Row>
      <Value column="id">2</Value>
      <Value column="description">robo por sorpresa</Value>
      <Value column="es_description">robo por sorpresa</Value>
    </Row>
  <Row>
      <Value column="id">3</Value>
      <Value column="description">robo con fuerza</Value>
      <Value column="es_description">robo con fuerza</Value>
    </Row>
  <Row>
      <Value column="id">4</Value>
      <Value column="description">robo de veículo</Value>
      <Value column="es_description">robo de veículo</Value>
    </Row>
  <Row>
      <Value column="id">5</Value>
  